# 数据准备

In [1]:
import numpy as np
import pandas as pd
# from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
byqxlsl='毕业去向落实率（%）'
lzl='离职率（%）'
xy='学院名称'
zy='专业名称'
ysr='月收入（元）'
zyxgd='专业相关度（%）'
jymyd='就业满意度（%）'
jxmyd='教学满意度（%）'
hxkczyd='核心课程重要度（%）'
hxkcmyd='核心课程满足度（%）'
jbgznlmzd='基本工作能力满足度（%）'
xytjd='校友推荐度（%）'

In [3]:
dict={
    byqxlsl:'就业率',
    lzl:'稳定率'
}

In [5]:
# 导入数据
data_ori = pd.read_excel(r'L:/项目配仓/兰州石化职业技术大学/二期/04模型及实施过程/就业跟踪反馈中心/兰州石化原始调研数据/兰州石化职业技术大学原始数据（2022届三个月大）/本校2022届主要指标表.xlsx', sheet_name='各专业主要指标表')
data_ori = data_ori.replace('—', np.nan)
data_ori = data_ori.fillna(data_ori.mean(numeric_only=True))
data_ori.insert(0, 'index', data_ori.index)
data_ori.head(2)

,index,学院名称,专业名称,毕业去向落实率（%）,月收入（元）,专业相关度（%）,就业满意度（%）,离职率（%）,教学满意度（%）,核心课程重要度（%）,核心课程满足度（%）,基本工作能力满足度（%）,校友推荐度（%）,校友满意度（%）,培养目标总体了解度（%）,毕业要求了解度（%）,就业岗位适应性（%）
0,0,本校平均,本校平均,85.63,5577.650000,70.240000,83.510000,37.440000,93.440000,89.830000,90.430000,96.54000,80.450000,94.380000,86.580000,96.090000,93.31000
1,1,资源环境工程系,工程测量技术（资源环境工程系）,76.19,5296.302121,65.964043,83.216279,40.006042,94.163962,85.581364,88.881951,96.85303,80.889286,94.643455,86.224464,95.768254,93.26475


# 第一部分算权重

In [6]:
data1 = data_ori[[byqxlsl,lzl,ysr,zyxgd,jymyd,jxmyd,hxkczyd,hxkcmyd,jbgznlmzd,xytjd]]
data1.head(2)

,毕业去向落实率（%）,离职率（%）,月收入（元）,专业相关度（%）,就业满意度（%）,教学满意度（%）,核心课程重要度（%）,核心课程满足度（%）,基本工作能力满足度（%）,校友推荐度（%）
0,85.63,37.440000,5577.650000,70.240000,83.510000,93.440000,89.830000,90.430000,96.54000,80.450000
1,76.19,40.006042,5296.302121,65.964043,83.216279,94.163962,85.581364,88.881951,96.85303,80.889286


In [7]:
# 归一化
min_max_model = MinMaxScaler()
data_f = min_max_model.fit_transform(data1)
d_columns = data1.columns
d_index = data1.index
data_f = pd.DataFrame(list(data_f), columns=d_columns, index=d_index)
# 离职率分数倒转
data_f[lzl] = 1-data_f[lzl]
data_f.head(2)

,毕业去向落实率（%）,离职率（%）,月收入（元）,专业相关度（%）,就业满意度（%）,教学满意度（%）,核心课程重要度（%）,核心课程满足度（%）,基本工作能力满足度（%）,校友推荐度（%）
0,0.632763,0.65598,0.510535,0.669333,0.523686,0.672000,0.804724,0.707875,0.606371,0.550220
1,0.391515,0.61643,0.437905,0.621823,0.515202,0.708198,0.723144,0.660621,0.641983,0.565076


In [8]:
# 之后的都看不懂，全弄一块
# 原始数据标准化并将数据类型转换成numpy.array
X = data_f.copy().values
s = StandardScaler()
X_s = s.fit_transform(X)

# 最初因子分析的载荷矩阵
fa_loadings = np.array([[ 0.25566256, -0.12154006,  0.05654762,  0.68626045],
                       [ 0.70960664, -0.18468978,  0.22449588, -0.08613011],
                       [ 0.74913007,  0.06034118, -0.15372229,  0.29158973],
                       [ 0.57740944,  0.5388749 ,  0.22938802, -0.07109893],
                       [-0.09470698, -0.06827397,  0.81577666,  0.19956862],
                       [ 0.03871277,  0.1284292 ,  0.09007327,  0.75591247],
                       [ 0.03682195,  0.67220926,  0.0088286 ,  0.31945562],
                       [-0.1727628 ,  0.80854032, -0.04960903, -0.21393947],
                       [ 0.22277429,  0.09196496,  0.80151372, -0.03718159],
                       [ 0.73425732, -0.0251303 ,  0.02961447,  0.35727926]])

# 计算权重，利用权重和标准化数据计算因子得分
structure = fa_loadings
corr_ = data_f.corr()
corr_1 = corr_.values
weights = np.linalg.solve(corr_1, structure)
scores = np.dot(X_s, weights)
scores

# 把各因子得分和年份、院系、专业结合
d_columns_1 = ['职业发展','工作能力','母校评价','就业能力']
data_score_0 = pd.DataFrame(list(scores),columns=d_columns_1, index=d_index)
data_score_0

# 计算各因子指数分
for i in list(data_score_0.columns):
    Max = np.max(data_score_0[i])
    Min = np.min(data_score_0[i])
    j = i + '评分'
    data_score_0[j] = round((data_score_0[i] - Min)/(Max - Min) * 100, 2)
data_score_0

weight = [0.323212105,0.228255387,0.223707081,0.224825426]

# 根据权重计算综合评分得分
score_0 = np.dot(np.array(data_score_0[data_score_0.columns[0:4]]), np.array(weight))
score_0

# 把综合评分得分加入数据框data_score
data_score_0['综合能力'] = score_0
data_score_0

# 计算并加入综合得分指数
max_score = np.max(data_score_0['综合能力'])
min_score = np.min(data_score_0['综合能力'])
data_score_0['综合得分'] = round((data_score_0['综合能力'] - min_score)/(max_score - min_score) * 100, 2)
data_score_0

,职业发展,工作能力,母校评价,就业能力,职业发展评分,工作能力评分,母校评价评分,就业能力评分,综合能力,综合得分
0,0.132379,0.413739,-0.338631,-0.018229,53.49,49.79,42.46,52.84,0.057372,52.51
1,0.263308,0.158143,-0.157908,-1.176791,55.23,46.38,44.85,36.97,-0.178696,46.88
2,0.519406,0.311955,-0.311492,-2.321360,58.62,48.43,42.82,21.29,-0.352500,42.73
3,-0.326682,-0.196205,0.195914,1.460025,47.40,41.66,49.51,73.09,0.221706,56.43
4,1.644504,-2.690446,0.360578,-0.787274,73.55,8.38,51.68,42.31,-0.178921,46.87
...,...,...,...,...,...,...,...,...,...,...
69,0.208577,0.125271,-0.125085,-0.932185,54.50,45.94,45.28,40.32,-0.141553,47.76
70,-0.550793,0.839070,-0.470335,-2.495740,44.43,55.47,40.73,18.90,-0.652824,35.56
71,-0.842721,1.195661,-0.819984,-0.456442,40.55,60.22,36.12,46.84,-0.285517,44.33
72,-0.513246,-0.229488,0.249590,1.395915,44.92,41.21,50.22,72.22,0.151403,54.76


In [9]:
# 加index用于合并
data_score_0.insert(0, 'index', data_score_0.index)

In [10]:
data_score_0

,index,职业发展,工作能力,母校评价,就业能力,职业发展评分,工作能力评分,母校评价评分,就业能力评分,综合能力,综合得分
0,0,0.132379,0.413739,-0.338631,-0.018229,53.49,49.79,42.46,52.84,0.057372,52.51
1,1,0.263308,0.158143,-0.157908,-1.176791,55.23,46.38,44.85,36.97,-0.178696,46.88
2,2,0.519406,0.311955,-0.311492,-2.321360,58.62,48.43,42.82,21.29,-0.352500,42.73
3,3,-0.326682,-0.196205,0.195914,1.460025,47.40,41.66,49.51,73.09,0.221706,56.43
4,4,1.644504,-2.690446,0.360578,-0.787274,73.55,8.38,51.68,42.31,-0.178921,46.87
...,...,...,...,...,...,...,...,...,...,...,...
69,69,0.208577,0.125271,-0.125085,-0.932185,54.50,45.94,45.28,40.32,-0.141553,47.76
70,70,-0.550793,0.839070,-0.470335,-2.495740,44.43,55.47,40.73,18.90,-0.652824,35.56
71,71,-0.842721,1.195661,-0.819984,-0.456442,40.55,60.22,36.12,46.84,-0.285517,44.33
72,72,-0.513246,-0.229488,0.249590,1.395915,44.92,41.21,50.22,72.22,0.151403,54.76


# 第二部分

In [11]:
data2 = data_ori[[ysr,zyxgd,jymyd,jbgznlmzd,lzl]]
data2.head(2)

,月收入（元）,专业相关度（%）,就业满意度（%）,基本工作能力满足度（%）,离职率（%）
0,5577.650000,70.240000,83.510000,96.54000,37.440000
1,5296.302121,65.964043,83.216279,96.85303,40.006042


In [12]:
d_columns = data2.columns
d_index = data2.index

# 归一化
min_max_model = MinMaxScaler()
data_f = min_max_model.fit_transform(data2)
data_f = pd.DataFrame(list(data_f), columns=d_columns, index=d_index)
data_f

weights = np.array([0.1030,0.1324,0.3219,0.1686,0.2741])

data_score_1 = pd.DataFrame(index=d_index)

data_score_1['薪资水平'] = round(data_f[ysr] * 100, 2)
data_score_1['专业对口度'] = round(data_f[zyxgd] * 100, 2)
data_score_1['满意度'] = round(data_f[jymyd] * 100, 2)
data_score_1['成长性'] = round(data_f[jbgznlmzd] * 100, 2)
data_score_1['稳定性'] = round(data_f[lzl] * 100, 2)

data_score_1['综合指标（五高）'] = np.dot(np.array(data_score_1[data_score_1.columns[0:5]]), np.array(weights))

max_score = np.max(data_score_1['综合指标（五高）'])
min_score = np.min(data_score_1['综合指标（五高）'])
data_score_1['综合评分（五高）'] = round((data_score_1['综合指标（五高）'] - min_score)/(max_score - min_score) * 100, 2)
# 加index用于合并
data_score_1.insert(0, 'index', data_score_1.index)
data_score_1

,index,薪资水平,专业对口度,满意度,成长性,稳定性,综合指标（五高）,综合评分（五高）
0,0,51.05,66.93,52.37,60.64,34.40,50.630529,38.62
1,1,43.79,62.18,51.52,64.20,38.36,50.665886,38.71
2,2,43.79,62.18,51.52,64.20,38.36,50.665886,38.71
3,3,43.79,62.18,51.52,64.20,38.36,50.665886,38.71
4,4,53.52,66.67,51.52,64.20,38.36,52.262552,42.91
...,...,...,...,...,...,...,...,...
69,69,43.79,62.18,51.52,64.20,38.36,50.665886,38.71
70,70,78.45,91.11,51.52,64.20,41.60,58.954282,60.49
71,71,60.40,58.33,51.52,64.20,77.22,62.518502,69.86
72,72,39.30,59.60,51.52,64.20,38.36,49.861824,36.60


# 合并

In [13]:
data_sum = data_ori.merge(data_score_0.merge(data_score_1,on='index'),on='index')
data_sum['index']=2022
# 改名
data_sum = data_sum.rename(columns={'index':'jb'})

# 有要除100的

list_100 = []
for col_name in list_100:
    data_sum[col_name] = round(data_sum[col_name]/100,2)

In [14]:
data_sum.head(2)

,jb,学院名称,专业名称,毕业去向落实率（%）,月收入（元）,专业相关度（%）,就业满意度（%）,离职率（%）,教学满意度（%）,核心课程重要度（%）,...,就业能力评分,综合能力,综合得分,薪资水平,专业对口度,满意度,成长性,稳定性,综合指标（五高）,综合评分（五高）
0,2022,本校平均,本校平均,85.63,5577.650000,70.240000,83.510000,37.440000,93.440000,89.830000,...,52.84,0.057372,52.51,51.05,66.93,52.37,60.64,34.40,50.630529,38.62
1,2022,资源环境工程系,工程测量技术（资源环境工程系）,76.19,5296.302121,65.964043,83.216279,40.006042,94.163962,85.581364,...,36.97,-0.178696,46.88,43.79,62.18,51.52,64.20,38.36,50.665886,38.71


# 保存至excel

In [15]:
data_sum.to_excel(r'E:/桌面/兰州石化因子分析得分结果(根据初始体系计算).xlsx')